<a href="https://colab.research.google.com/github/kmnsky/DS_Homework/blob/master/Daily_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Daily stat**

In [110]:
# data analysis and wrangling
import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Mounting GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

# Writing multiple dataframes to worksheets using Pandas and XlsxWriter
!pip install xlsxwriter
import xlsxwriter

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
xls = pd.ExcelFile("/content/drive/My Drive/Datasets/DE_10-00.xlsx")
df1 = pd.read_excel(xls, 'Information')

In [112]:
xls = pd.ExcelFile("/content/drive/My Drive/Datasets/DE_11-00.xlsx")
df2 = pd.read_excel(xls, 'Information')

## **Hourly Analitics**

In [113]:
result = pd.merge(df2, df1, how="left", on=["Utm_campaign"], suffixes=('_x', ''))

In [114]:
# Замена NaN
result.fillna(0, inplace=True)

In [115]:
# Вычисление
result['Cost_x'] = result['Cost_x'] - result['Cost']
result['Show_x'] = result['Show_x'] - result['Show']
result['Link_cliks_x'] = result['Link_cliks_x'] - result['Link_cliks']
result['Install_x'] = result['Install_x'] - result['Install']
result['Reg_x'] = result['Reg_x'] - result['Reg']
result['Ftd_x'] = result['Ftd_x'] - result['Ftd']

In [116]:
result.drop(['Time', 'Cost', 'Show', 'Link_cliks', 'Install', 'Reg',
       'Ftd', 'adset', 'geo', 'brand', 'page', 'soc', 'bm', 'acc', 'camp',
       'adset1', 'int', 'placement', 'gender', 'age', 'text', 'creo', 'app',
       'os', 'mb'], axis=1, inplace = True)

In [117]:
result.columns = ['Time', 'Utm_campaign', 'Cost', 'Show', 'Link_cliks', 'Install', 'Reg',
       'Ftd', 'adset', 'geo', 'brand', 'page', 'soc', 'bm', 'acc', 'camp',
       'adset1', 'int', 'placement', 'gender', 'age', 'text', 'creo', 'app',
       'os', 'mb']

In [118]:
df_final = df1.append(result.loc[:, :])

### **Group by**

In [119]:
# Группировка при выборке df_final
df_group = pd.DataFrame(df_final.groupby(['bm'], as_index=False).sum().sort_values(['Cost'], ascending=False))

In [120]:
df_group["ID"] = np.arange(0, 0 + len(df_group))

In [121]:
df_group = df_group[['bm', 'ID']]

In [122]:
# Добавление столбца ID для сортировки
df_final = pd.merge(df_final, df_group, how="left", on=["bm"])

In [123]:
# Сводная таблица
df_pivot = df_final.groupby(['ID', 'bm']).apply(lambda sub: sub.pivot_table(
    index=['Utm_campaign', 'Time'],
    values=['Cost', 'Install', 'Show', 'Link_cliks', 'Reg', 'Ftd'],
    aggfunc=np.sum,
    margins=True,
    margins_name='Total'
))

In [124]:
# Вычисление
df_pivot = df_pivot.assign(Instal_cost = df_pivot.Cost/df_pivot.Install)
df_pivot = df_pivot.assign(Reg_cost = df_pivot.Cost/df_pivot.Reg)
df_pivot = df_pivot.assign(FTD_cost = df_pivot.Cost/df_pivot.Ftd)
df_pivot = df_pivot.assign(CTR = df_pivot.Link_cliks/df_pivot.Show*100)
#df_pivot.loc[('Total', '', '')] = df_pivot.sum()

In [125]:
# Замена Inf
df_pivot.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

In [126]:
# Два знака после запятой
df_pivot = df_pivot.round(decimals=2)

In [127]:
# Удалить колонки 
df_pivot = df_pivot.drop(['Link_cliks', 'Show'], axis=1)

In [129]:
# Сортировка по первому уровню
df_pivot = df_pivot.sort_values(by = ["ID", 'Utm_campaign'], axis = 0, ascending=[True, False])

In [130]:
df_pivot = df_pivot.droplevel('ID')

In [133]:
# Загрузить файл
df_pivot.to_excel('AdsetPredicted.xlsx')
from google.colab import files
files.download('AdsetPredicted.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [134]:
writer = pd.ExcelWriter('DE_10-00.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_pivot.to_excel(writer, sheet_name='Total_DE')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet_total = writer.sheets['Total_DE']

# Format all the columns.
my_format = workbook.add_format()
my_format.set_align('center')

# Set the column width and format.
worksheet_total.set_column('D:K', None, my_format)
worksheet_total.set_column('A:A', 18)
worksheet_total.set_column('B:B', 50)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [135]:
# Загрузить файл
from google.colab import files
files.download('DE_10-00.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>